<a href="https://colab.research.google.com/github/chandanareddy606/Gender_Detection/blob/branch/Webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image

In [ ]:
from tensorflow.keras.models import load_model
import os
import io
import cv2

In [ ]:
!pip install git+git://github.com/PnS2019/pnslib.git

In [ ]:
from pnslib import utils

In [ ]:
model=load_model('/content/drive/My Drive/Cnn/Gender_Detector.model')
model.summary()

In [ ]:
model.layers

In [ ]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
            
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
        

    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x



In [ ]:
labels_dict={0:'man',1:'woman'}  ### should be changed according to your model
color_dict={0:(0,255,0),1:(0,0,255)}

size = 5
VideoCapture()
eval_js('create()')#Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier(utils.get_haarcascade_path('haarcascade_frontalface_default.xml'))

while True:
    byte = eval_js('capture()')
    frame = byte2image(byte)
   
    # Resize the image to speed up detection
    mini = cv2.resize(frame, (frame.shape[1] // size, frame.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        face_img=frame[y:y+h,x:x+w]     #### frame size of the detection
        resized=cv2.resize(face_img,(224,224))   ## Resizing the image
        normalized=resized/255.0        #### should be same as the input to conv2d layers other wise throws error
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],4)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    # Show the image
    eval_js('showimg("{}")'.format(image2byte(frame)))
    key = cv2.waitKey(1)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break

eval_js('create()').release()    ### have not figured out how to stop a cam when inactive after sometime , so it keep on untill u stop this cell from executing

# Close all started windows
cv2.destroyAllWindows()